# Setup

In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
import httpx, os, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from urllib.parse import urlparse
import phoenix as px
from phoenix.experiments import run_experiment

from search_lib.load_data import download_sqlite_file, get_notes
from search_lib.search import sparse_search, dense_search, hybrid_search
from search_lib.evals import average_precision_evaluator, r_precision_evaluator, nrbp_evaluator
import search_lib.evals
from search_lib.pre_search import keyword_expansion

In [3]:
from search_lib.pre_search import decompose_query
from search_lib.post_search import cohere_rerank
from functools import partial
from fastcore.parallel import parallel

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
db_path = download_sqlite_file()
db_path

Path('AnKing Step Deck.sqlite')

In [6]:
notes = get_notes(db_path)
notes[0]

Extracted 28664 notes with content


{'id': '10d7cd51-565b-44d6-9858-30cb6ebf51c4',
 'content': 'For instructions on <b>using this deck</b>&nbsp;and&nbsp;<b>using/customizing the note type</b>&nbsp;go to <a href="https://community.ankihub.net/t/wiki-anking-overhaul-for-step-1-2-by-ankingmed/114092?_gl=1*15fqpnu*_ga*MTA1OTEyNjc5Ni4xNjkxODA1MTI5*_ga_T2ZF93TKF6*MTcwNzQxMzQwOS45OTcuMS4xNzA3NDE5NTEwLjAuMC4w">the wiki guide</a><br><br>{{c1::Please note that all organization for this deck is in the <b>tags</b>! There are <u><a href="https://community.ankihub.net/t/why-are-there-no-subdecks-in-the-anking-v12-deck/129271">no subdecks</a></u>}}<br><br>If you are wondering why there are <u>0 cards showing up</u>, see this <a href="https://community.ankihub.net/t/why-does-my-anking-step-deck-have-0-cards/131517">post for the answer</a> Huge thank you to all the&nbsp;<a href="https://community.ankihub.net/docs?topic=134040">contributors &amp; maintainers</a>!&nbsp;<br>To see what is being updated, check the&nbsp;<a href="https://commu

In [45]:
notes[0]['tags']

['AnkiHub_Subdeck::AnKing_Overhaul_for_Step_1_&_2::Instructions',
 'AnkiHub_Instructions']

In [7]:
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://phx.ankihub.net"

In [8]:
client = px.Client()
ds = client.get_dataset(name="Smart search golden dataset")

In [9]:
ds.as_dataframe().head(2)

,input,output,metadata
example_id,,,
RGF0YXNldEV4YW1wbGU6Nzk=,{'query': 'propofol'},"{'documents': [{'rating': 1.0, 'anki_id': 1523...",{}
RGF0YXNldEV4YW1wbGU6ODA=,{'query': 'B. Cysticfibrosis CF is the most co...,"{'documents': [{'rating': 1.0, 'anki_id': 1473...",{}


# Search

Let's create a notebook that demonstrates how to use Phoenix to run a search experiment using BM25 sparse search. We'll go through the entire process from getting the dataset to evaluating the results.

In [10]:
def bm25_search_experiment(query, notes, top_k=400):
    """Search function that will be used in the Phoenix experiment"""
    results = sparse_search(query, notes, top_k=top_k, verbose=False)
    formatted_results = []
    for result in results:
        formatted_results.append({
            "anki_id": result["id"], 
            "content": result["content"],
            "similarity": result["similarity"],
            "tags": result["tags"]})    
    return formatted_results

In [11]:
import pickle
notes_content = [n['content'] for n in notes]
embeddings_fpath = 'cohere_embeddings.pkl'
if os.path.exists(embeddings_fpath):
    with open(embeddings_fpath, 'rb') as f:
        embeddings = pickle.load(f)
else: 
    embeddings = embed_cohere(notes_content, verbose=True, input_type='search_document')
    with open(embeddings_fpath, 'wb') as f:
        pickle.dump(embeddings, f)

In [12]:
def semantic_search_experiment(query, notes, top_k=400):
    """Search function that will be used in the Phoenix experiment"""
    results, _ = dense_search(query, notes, top_k=top_k, embeddings=embeddings, verbose=False)
    formatted_results = []
    for result in results:
        formatted_results.append({
            "anki_id": result["id"], 
            "content": result["content"],
            "similarity": result["similarity"],
            "tags": result["tags"]})    
    return formatted_results

## Exploration

Look at an example of running the bm25_search_experiment function on a query.

**Important**: This is where we'd iterate with lots of queries and ideas to get an idea of what kinds of queries seem to make sense to us vs which ones do not.  This may lead to questions to ask a domain expert.  Try lots of stuff, before looking at any metrics.

### Individual Queries

In [13]:
query = "What causes cystic fibrosis?"
search_results = semantic_search_experiment(query, notes)
for i, result in enumerate(search_results[:3]):
    print(f"\n--- Result {i+1} : {result['anki_id']} ---\nSimilarity: {result['similarity']:.4f}\n{result['content'][:150]}")


--- Result 1 : 1b0071a5-9ae3-4c6c-8063-20ba6e83c7eb ---
Similarity: 0.6339
<div><b>Cystic fibrosis</b> may cause {{c1::<u>pancreatic</u>}}<u> insufficiency</u>, resulting in <i>malabsorption</i></div> <img src="57415c3098c611

--- Result 2 : 85f19d50-8020-42bc-bafb-681e64db7e72 ---
Similarity: 0.6276
<b>Cystic fibrosis</b> is due to a(n) {{c1::autosomal recessive::inheritance pattern}} <u>CFTR</u> defect; commonly seen in people of {{c2::Northern E

--- Result 3 : 27bd0932-a92c-4927-b62b-13c3333ce5b5 ---
Similarity: 0.6000
<div><b>Cystic fibrosis </b>most commonly occurs due to a(n) {{c2::<u>in-frame</u>}} <u>deletion</u> of {{c1::Phe508}}</div> - This results in an <b>i


In [14]:
query = "What causes cystic fibrosis?"
search_results = bm25_search_experiment(query, notes)
for i, result in enumerate(search_results[:3]):
    print(f"\n--- Result {i+1} : {result['anki_id']} ---\nSimilarity: {result['similarity']:.4f}\n{result['content'][:150]}")

Split strings:   0%|          | 0/28664 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/28664 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/28664 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]


--- Result 1 : c187a291-4102-40e7-8ef0-38b30665537a ---
Similarity: 1.0000
<div><u>Increased</u> <b>Na<sup>+</sup> reabsorption</b> in cystic fibrosis causes a more {{c1::<i>negative</i>}} <b>transepithelial potential differe

--- Result 2 : b6b81223-3f02-457f-9e28-45c6a37d1fba ---
Similarity: 0.9799
<div><b>Cystic fibrosis</b> may present with {{c1::<b>contraction</b>::volume}}<b> </b>{{c2::<b>alkalosis</b>::pH}} </div> - Inability to reabsorb Cl<

--- Result 3 : 5b5c9996-b3ff-4c6b-af95-2b05f7b3976b ---
Similarity: 0.9749
What is the likely <i>underlying etiology </i>in a patient with <b>bronchiectasis</b> predominantly in the <u>upper lobe</u> and sputum cultures posit


In [15]:
query = "Brain"
search_results = bm25_search_experiment(query, notes)
for i, result in enumerate(search_results[:3]):
    print(f"\n--- Result {i+1} : {result['anki_id']} ---\nSimilarity: {result['similarity']:.4f}\n{result['content'][:150]}")

Split strings:   0%|          | 0/28664 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/28664 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/28664 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]


--- Result 1 : c11b1d6d-61be-4792-85fe-d2fb5b84a422 ---
Similarity: 1.0000
<b>Herniation</b> is displacement of <u>brain tissue</u> due to {{c1::mass effect}} or increased {{c2::intracranial pressure}} <img src="35ce0a50b8a65

--- Result 2 : ebdacb43-05f4-4c6d-b8c5-4ae865b1fe0d ---
Similarity: 0.9891
Head trauma = {{c1::CT::imaging}} Basilar skull fracture, hematomas, axonal injuries, concussion, etc<br><br><img src="3c325400bea8f34bb1757845d960a8f

--- Result 3 : 8b038a92-af84-4096-9ccc-2b29b246e56c ---
Similarity: 0.9872
<b>Acoustic neuromas</b> are <u>more common</u> in {{c1::elderly patients::population}} Median age is 50 years     <img src="paste-4e64d9961d933e539ca


In [16]:
# Let's try a more complex, longer query about a different medical topic
query = """
Explain the pathophysiology of myocardial infarction in detail, including the role of atherosclerosis, 
thrombosis, and the cascade of events that leads to myocardial cell death. Also include information about 
risk factors such as hypertension, diabetes, smoking, and hyperlipidemia. What are the typical ECG changes 
seen in different types of myocardial infarction (STEMI vs NSTEMI)? How do troponin levels change over time 
after an MI? What are the complications of myocardial infarction including arrhythmias, heart failure, 
cardiogenic shock, and mechanical complications like papillary muscle rupture? Finally, discuss the 
management strategies including primary PCI, thrombolytics, antiplatelet therapy, beta-blockers, ACE 
inhibitors, and statins.
"""

search_results = bm25_search_experiment(query, notes, top_k=5)
for i, result in enumerate(search_results[:3]):
    print(f"\n--- Result {i+1} : {result['anki_id']} ---\nSimilarity: {result['similarity']:.4f}\n{result['content'][:150]}")

Split strings:   0%|          | 0/28664 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/28664 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/28664 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]


--- Result 1 : 6ff38b1a-9d48-4a08-b5c5-e2b2874e5085 ---
Similarity: 1.0000
- <b>Papillary muscle rupture</b> will present with findings of {{c1::left}} heart failure<br><br>

- <b>Interventricular septum rupture </b>will pres

--- Result 2 : 8b42ea92-75cb-4bee-82d4-fcf00263d4cf ---
Similarity: 0.9662
What is the <u>next step in diagnosis</u> for a patient with new-onset <b>epigastric pain,</b> <b>nausea</b>, <b>hypotension,</b> <b>bradycardia </b>a

--- Result 3 : e403c234-ce6e-4dba-97e6-c9db5e5ffffb ---
Similarity: 0.9186
<div>One complication that may occur during the <b>first 24 hours</b> post-MI is {{c1::arrhythmia}}, which is an <b>important cause of death</b> befor


In [17]:
instructor_readme = httpx.get('https://raw.githubusercontent.com/567-labs/instructor/refs/heads/main/README.md').text

In [18]:
# llm_rerank(query, search_results)

In [19]:
import instructor
from pydantic import BaseModel, Field
import google.generativeai as genai
from concurrent.futures import ThreadPoolExecutor

def llm_rerank(query, notes, limit=None, model_name="gemini-1.5-flash"):
    """Use an LLM to rerank notes based on relevance to the query."""
    class RelevanceScore(BaseModel):
        score: float = Field(description="Relevance score from 0-1")
    
    # Initialize the model
    model = genai.GenerativeModel(model_name)
    
    # Based on the instructor README, for Gemini models we should use from_gemini
    client = instructor.from_gemini(
        client=model,
        mode=instructor.Mode.GEMINI_JSON  # Use GEMINI_JSON mode for Gemini models
    )
    
    def process_note(note):
        # Extract content from the note
        text = note['content']
        resp = client.chat.completions.create(
            response_model=RelevanceScore,
            messages=[
                {"role": "user", "content": f"Query: {query}\n\nDocument: {text}\n\nRate how relevant this document is to the query from 0.0 (not relevant) to 1.0 (highly relevant). Just return the score."}
            ]
        )
        note_copy = note.copy()
        note_copy["relevance_score"] = resp.score
        return note_copy
    
    # Parallel processing function
    def parallel(func, items, max_workers=20):
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(executor.map(func, items))
        return results
    
    results = [r for r in parallel(process_note, notes) if r is not None]
    results.sort(key=lambda x: x.get("relevance_score", 0), reverse=True)
    
    if limit: 
        return results[:limit]
    return results

### Documents

In [20]:
_file_qrys = [
    'https://app.ankihub.net/ai/file-search/input-file/ai-files/193108/8feac2bc-f598-4b8e-a8ad-2d6684848e03.pdf/',
    'https://app.ankihub.net/ai/file-search/input-file/ai-files/83793/4f171ac2-3e61-4ab4-a531-64c9455505c7.pdf/',
    'https://app.ankihub.net/ai/file-search/input-file/ai-files/83793/e4984b18-e7cb-48c3-a03e-3cd5cb36bd96.pdf/',
    'https://app.ankihub.net/ai/file-search/input-file/ai-files/83793/29627a54-579d-4ced-978d-9b6f4b5c59af.pdf/',
    'https://app.ankihub.net/ai/file-search/input-file/ai-files/83793/7a4d6de2-280a-4418-b6a8-55dadcb4fc72.pdf/',
    'https://drive.google.com/file/d/1xLJik0uCLOzS59oEUtMlUf36z0evAEVU/view?usp=sharing']

In [21]:
from search_lib.pre_search import process_pdf

In [22]:
from IPython.display import Markdown
# Markdown(process_pdf(_file_qrys[0]))

## Targetted Metrics

Once you feel like you have a good idea of how this works on different types of queries, we can calculate metrics using the annotated dataset against particular queries, or all of them.

This allows you to keep trying to iterate on known problems quickly without re-running the full experiments.

In [23]:
df = ds.as_dataframe()
def categorize_query(query):
    if 'http' in query.lower(): return "URL"
    word_count = len(query.split())
    if word_count < 5: return "keyword"
    return "mid-length text"
df['query_type'] = df['input'].apply(lambda x: categorize_query(x['query']))

In [24]:
query = df.input.iloc[1]['query']
expected_output = df.output.iloc[1]
search_results = bm25_search_experiment(query, notes)

formatted_output = {"documents": search_results}

nrbp_score = nrbp_evaluator(expected_output, formatted_output)
r_precision = r_precision_evaluator(expected_output, formatted_output)
avg_precision = average_precision_evaluator(expected_output, formatted_output)

print(f"Query: '{query[:100]}...'")
print(f"nRBP score: {nrbp_score:.4f}")
print(f"R-Precision score: {r_precision:.4f}")
print(f"Average Precision score: {avg_precision:.4f}")

Split strings:   0%|          | 0/28664 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/28664 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/28664 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Query: 'B. Cysticfibrosis
CF is the most common lethal genetic disease in Caucasians of Northern European an...'
nRBP score: 0.6234
R-Precision score: 0.5000
Average Precision score: 0.3944


When we're ready at the end and we feel good that this is an improvement, then we can run the full thing as a phoenix experiment to get all the metrics and see how it does.

## Improving

In [25]:
exp = client.get_experiment(experiment_id='RXhwZXJpbWVudDo1MzA=')

In [26]:
def format_cohere_results(cohere_response, original_results):
    content_to_result = {r["content"]: r for r in original_results}

    formatted_results = []
    for item in cohere_response.results:
        original = content_to_result.get(item.document.text)
        if original:
            # Create a new dict with the original data but updated similarity
            formatted_results.append({
                "anki_id": original["anki_id"],
                "content": original["content"],
                "tags": original.get("tags", []),
                "similarity": item.relevance_score  # Use the reranked score
            })
    
    return formatted_results

In [27]:
top_k = 200
def search_task(input):
    query = input.get('query', '')
    print(query)
    if query.startswith('http'):
        query = process_pdf(query)
    if len(query.split(' ')) > 6:
        queries = decompose_query(query).queries
    else:
        queries = [f"{query} {query} {keyword_expansion(query)}"]
#         print(f"{queries=}")
    _bm25_search_experiment = partial(bm25_search_experiment, notes=notes, top_k=top_k//2)
    results = parallel(_bm25_search_experiment, queries)
# Flatten results and deduplicate based on anki_id
    flattened_results = []
    seen_ids = set()
    for sublist in results:
        for item in sublist:
            if item["anki_id"] not in seen_ids:
                seen_ids.add(item["anki_id"])
                # Ensure we have a standardized structure
                flattened_results.append({
                    "anki_id": item["anki_id"],
                    "content": item["content"],
                    "tags": item.get("tags", []),
                    "similarity": item.get("similarity", 0)
                })
    reranked_response = cohere_rerank(query, flattened_results)
    results = format_cohere_results(reranked_response, flattened_results)
    return {
        'documents': results,
        'metadata': {
            'scoring_scale': '',
            'top_k': top_k,
        }
    }

# res = search_task({'query': "The heart has four chambers: right atrium, right ventricle, left atrium, and left ventricle. Blood flows through these chambers in a specific pattern. The right atrium receives deoxygenated blood from the body via the superior and inferior vena cava. This blood then flows through the tricuspid valve into the right ventricle, which pumps it to the lungs through the pulmonary artery."})

In [28]:
bad_performing_ids = [
    'RGF0YXNldEV4YW1wbGU6MTgwMw==', 'RGF0YXNldEV4YW1wbGU6MTgwMg==', 'RGF0YXNldEV4YW1wbGU6MTY4NQ==', 'RGF0YXNldEV4YW1wbGU6ODQ=', 'RGF0YXNldEV4YW1wbGU6ODI=', 'RGF0YXNldEV4YW1wbGU6MTc5Nw==', 'RGF0YXNldEV4YW1wbGU6MTc5Ng==',]

In [33]:
# First, get the query and expected output
# results = []
# for idx in df[df.query_type=='keyword'].index:
#     _row = dict(df[df.index == idx])
#     query = _row['input'][idx]['query']
#     expected_output = _row['output'][idx]
#     res = search_task({'query': query})
#     results.append({
#         'idx': idx,
#         'query': query[:100],
#         'nrbp_score': nrbp_evaluator(expected_output, res),
#         'r_precision': r_precision_evaluator(expected_output, res),
#         'avg_precision': average_precision_evaluator(expected_output, res)})
# pd.DataFrame(results)

propofol
queries=['propofol propofol intravenous anesthetic, sedative-hypnotic, general anesthesia, induction agent, ICU sedation, Diprivan']
Coenzyme Q
queries=['Coenzyme Q Coenzyme Q Coenzyme Q CoQ10 Ubiquinone Mitochondrial Coenzyme Q10 Antioxidant Vitamin Q10 Bioenergetics Mitochondrial respiratory chain ATP production Energy metabolism Lipid-soluble antioxidant Biochemical reactions Oxidative phosphorylation Electron transport chain Mitochondrial function Mitochondrial diseases Mitochondrial disorders Mitochondrial myopathy Mitochondrial encephalopathy Lactic acidosis Stroke-like episodes MELAS Syndrome Kearns-Sayre Syndrome Leigh Syndrome Mitochondrial DNA']
downstream PDH
queries=['downstream PDH downstream PDH downstream pyruvate dehydrogenase, PDH complex, pyruvate dehydrogenase complex, pyruvate dehydrogenase enzyme, pyruvate dehydrogenase deficiency, pyruvate dehydrogenase kinase, pyruvate dehydrogenase phosphatase']
Iga Vasculitis
queries=['Iga Vasculitis Iga Vasculitis IgA

,idx,query,nrbp_score,r_precision,avg_precision
0,RGF0YXNldEV4YW1wbGU6Nzk=,propofol,1.000000,1.000000,1.000000
1,RGF0YXNldEV4YW1wbGU6ODE=,Coenzyme Q,1.000000,1.000000,1.000000
2,RGF0YXNldEV4YW1wbGU6ODI=,downstream PDH,0.140456,0.083333,0.147611
3,RGF0YXNldEV4YW1wbGU6ODM=,Iga Vasculitis,0.951680,0.785714,0.817216
4,RGF0YXNldEV4YW1wbGU6ODQ=,Biochemestry lipids,0.944207,0.433333,0.345386
5,RGF0YXNldEV4YW1wbGU6MTgwNA==,STATIN INDUCED MYOPATHY,0.609710,0.600000,0.415833


In [ ]:
keyword_expansion('downstream PDH')

# Run Experiment

In [ ]:
queries = decompose_query("The heart has four chambers: right atrium, right ventricle, left atrium, and left ventricle. Blood flows through these chambers in a specific pattern. The right atrium receives deoxygenated blood from the body via the superior and inferior vena cava. This blood then flows through the tricuspid valve into the right ventricle, which pumps it to the lungs through the pulmonary artery.").queries

In [ ]:
# _bm25_search_experiment = partial(bm25_search_experiment, notes=notes, top_k=10)
# results = parallel(_bm25_search_experiment, queries)

In [ ]:
top_k = 200
# def search_task(input):
#     query = input.get('query', '')
#     if query.startswith('http'):
#         query = process_pdf(query)
#     queries = decompose_query(query).queries
#     _bm25_search_experiment = partial(bm25_search_experiment, notes=notes, top_k=top_k//2)
#     results = parallel(_bm25_search_experiment, queries)
#     # Flatten results and deduplicate based on anki_id
#     flattened_results = []
#     seen_ids = set()
#     for sublist in results:
#         for item in sublist:
#             if item["anki_id"] not in seen_ids:
#                 seen_ids.add(item["anki_id"])
#                 flattened_results.append(item)
                
# #     print(len(flattened_results))
# #     results = llm_rerank(query, results)
#     return {
#         'documents': results,
#         'metadata': {
#             'scoring_scale': '',
#             'top_k': top_k,
#         }
#     }
# res = search_task({'query': "The heart has four chambers: right atrium, right ventricle, left atrium, and left ventricle. Blood flows through these chambers in a specific pattern. The right atrium receives deoxygenated blood from the body via the superior and inferior vena cava. This blood then flows through the tricuspid valve into the right ventricle, which pumps it to the lungs through the pulmonary artery."})

In [ ]:

res = search_task({'query': _file_qrys[0]})

In [34]:
def run_bm25_experiment(dataset, # Phoenix dataset to use
                        top_k=200, # Number of top results to return
                       ): # The experiment results
    """Run a BM25 search experiment on a Phoenix dataset with multiple evaluation metrics"""

    evaluators = {
        "nrbp": nrbp_evaluator,
        "r_precision": r_precision_evaluator,
        "average_precision_at_200": average_precision_evaluator
    }
    
    return run_experiment(
        dataset=dataset,
        task=search_task,
        evaluators=evaluators,
        experiment_name=f"BM25 (top_{top_k}) - decomposition + rerank",
        experiment_description=f"Testing a full search task MVP with basic decomp/rerank",
        dry_run=False,  # Set to True for testing
        concurrency=1,
        timeout=900,
        print_summary=True)

In [35]:
experiment = run_bm25_experiment(dataset=ds, top_k=200)

🧪 Experiment started.
📺 View dataset experiments: https://phx.ankihub.net/datasets/RGF0YXNldDoxOA==/experiments
🔗 View this experiment: https://phx.ankihub.net/datasets/RGF0YXNldDoxOA==/compare?experimentId=RXhwZXJpbWVudDo1MzE=


running tasks |          | 0/20 (0.0%) | ⏳ 00:00<? | ?it/s

propofol
queries=['propofol propofol propofol anesthesia sedative hypnotic intravenous drug general anesthesia GABA receptor ICU sedation induction maintenance drug-induced sedation endoscopy sedation anesthesia induction anesthesia maintenance anesthesia drug anesthesia sedation drug anesthesia medication diprivan diprivan anesthesia diprivan sedation diprivan drug diprivan medication diprivan anesthesia drug diprivan sedative diprivan hypnotic diprivan general anesthesia diprivan GABA receptor diprivan ICU sedation diprivan induction dipr']


/Users/iflath/.venv/lib/python3.12/site-packages/phoenix/utilities/client.py:40: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


Retries exhausted after 1 attempts: Server error '502 Bad Gateway' for url 'https://phx.ankihub.net/v1/experiments/RXhwZXJpbWVudDo1MzE=/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502
B. Cysticfibrosis
CF is the most common lethal genetic disease in Caucasians of Northern European ancestry and has a prevalence of ∼1:3,300 births in the United States. CF is an autosomal-recessive disorder caused by mutations to the gene for the CF transmembrane conductance regulator (CFTR) protein that functions as a chloride channel on epithelium in the pancreas, lungs, testes, and sweat glands. Defective CFTR results in decreased secretion of chloride and increased uptake of sodium and water. In the pancreas, the depletion of water on the cell surface results in thickened mucus that clogs the pancreatic ducts, preventing pancreatic enzymes from reaching the intestine, thereby leading to pancreatic insufficiency. Treatment includes replacement of these enz

...

### I. **Embryology of the Integument**\n\n#### 1. **Origin and Initiation of Development**  \nThe integument originates primarily from the superficial ectoderm, which, during the early weeks of gestation (around the 3rd to 4th week), forms a single layer that will later develop into the epidermis. This differentiation is induced by signals from the adjacent mesoderm, which contributes to the formation of the dermis, demonstrating an essential interaction between these two tissues.  \n\n#### 2. **Differentiation and Maturation Processes**  \nDuring the embryonic phase, the epithelium undergoes differentiation into layers, initially forming the periderm and later establishing the basal layer, which gives rise to the stratified layers of the epidermis.  \nThrough molecular signals (such as those from the BMP, Wnt, and FGF families), induction and organization of skin appendages, such as hair follicles, sweat glands, and sebaceous glands, occur through invaginations and epithelial-mesench

https://app.ankihub.net/ai/file-search/input-file/ai-files/83793/7a4d6de2-280a-4418-b6a8-55dadcb4fc72.pdf/
https://app.ankihub.net/ai/file-search/input-file/ai-files/83793/29627a54-579d-4ced-978d-9b6f4b5c59af.pdf/
https://app.ankihub.net/ai/file-search/input-file/ai-files/83793/e4984b18-e7cb-48c3-a03e-3cd5cb36bd96.pdf/
https://app.ankihub.net/ai/file-search/input-file/ai-files/83793/4f171ac2-3e61-4ab4-a531-64c9455505c7.pdf/
After completing this brick, you will be able to:

Define appetite and satiety, hyperphagia and aphagia, and the dietary Calorie.
List the two major systems of the body involved in weight management and the concept of anorexigenic and orexigenic compounds.
Describe the role of appetite and metabolic rate in the maintenance of long-term energy, balance, and fat storage, and list and discuss the three normal components of energy expenditure.
List the anorexigenic hormones and the one known orexigenic hormone and their main secretagogue or mode of release, and briefly 

https://app.ankihub.net/ai/file-search/input-file/ai-files/193108/8feac2bc-f598-4b8e-a8ad-2d6684848e03.pdf/
Shape: Single, circular, double-stranded, and supercoiled (like a twisted rubber band). 

Supercoiling Enzyme: DNA gyrase – Targeted by Quinolones (antibiotics). 

Location: Attached to the inner membrane (not free-floating) 

No histones or nuclear membrane → enables fast replication (e.g., E. coli: ~20 min; C. perfringens: ~10-12 min). 

Transcription and translation occur simultaneously → increases replication speed. 

Ribosomes: 70S (50S + 30S) → targeted by aminoglycosides, tetracyclines, chloramphenicol, macrolides (erythromycin)  

2. A 10-year-old boy who was adopted from the Democratic Republic of Congo 2 weeks ago is brought to the physician for an initial examination. He appears slim, has thin extremities, and is in no distress. His temperature is 37.3°C (99.1 °F), pulse is 100/min, respirations are 20/min, and blood pressure is 118/68 mm Hg. The lungs are clear, and h

running experiment evaluations |          | 0/57 (0.0%) | ⏳ 00:00<? | ?it/s


🔗 View this experiment: https://phx.ankihub.net/datasets/RGF0YXNldDoxOA==/compare?experimentId=RXhwZXJpbWVudDo1MzE=

Experiment Summary (05/18/25 08:05 PM -0400)
--------------------------------------------
| evaluator                |   n |   n_scores |   avg_score |
|:-------------------------|----:|-----------:|------------:|
| average_precision_at_200 |  20 |         19 |    0.350844 |
| nrbp                     |  20 |         19 |    0.544222 |
| r_precision              |  20 |         19 |    0.387679 |

Tasks Summary (05/18/25 08:05 PM -0400)
---------------------------------------
|   n_examples |   n_runs |   n_errors |
|-------------:|---------:|-----------:|
|           20 |       19 |          0 |


In [ ]:
def run_semantic_search_experiment(dataset, # Phoenix dataset to use
                                  top_k=200, # Number of top results to return
                                 ): # The experiment results
    """Run a semantic search experiment on a Phoenix dataset with multiple evaluation metrics"""

    def search_task(input):
        query = input.get('query', '')
        if query.startswith('http'):
            query = process_pdf(query)
                
        results = semantic_search_experiment(query, notes, top_k=top_k)
        return {
            'documents': results,
            'metadata': {
                'scoring_scale': '0-2 (0=Not Relevant, 1=Somewhat Relevant, 2=Highly Relevant)',
                'top_k': top_k,
            }
        }
    
    # Create evaluators dictionary with all three metrics
    evaluators = {
        "nrbp": nrbp_evaluator,
        "r_precision": r_precision_evaluator,
        "average_precision_at_200": average_precision_evaluator
    }
    
    return run_experiment(
        dataset=dataset,
        task=search_task,
        evaluators=evaluators,
        experiment_name=f"Semantic Search (top_{top_k})",
        experiment_description=f"Testing semantic search with top {top_k} results, evaluated with multiple metrics (nRBP, R-Precision, Average Precision)",
        dry_run=False,  # Set to True for testing
        concurrency=1,
        timeout=900,
        print_summary=True)

You can run this experiment with:

In [ ]:
semantic_experiment = run_semantic_search_experiment(dataset=ds, top_k=200)